Imported Libaries

In [2]:
import os
from sqlalchemy import create_engine, inspect

import inspect
from IPython.display import display as original_display

import pandas as pd
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

# import geopandas as gpd
# from geopy.geocoders import Nominatim

import numpy as np
# import seaborn as sns

# import matplotlib.pyplot as plt
# import matplotlib.ticker as mtick
# import matplotlib.colors as mcolors

import re
# import string

import warnings
warnings.filterwarnings('ignore')

Script Specifc Imported Libaries

In [3]:
from tqdm import tqdm
import itertools

import threading
from concurrent.futures import ThreadPoolExecutor

from queue import Queue
results_queue = Queue()



import nltk
nltk_data_path = os.getenv('', 'default/path')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

# from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import spacy
from spellchecker import SpellChecker
# from wordsegment import load, segment
from wordsegment import Segmenter
from collections import Counter
# from wordcloud import WordCloud

# Initialize external tools
spell = SpellChecker()
custom_segmenter = Segmenter()
custom_segmenter.load()  # Load data for wordsegment
lemmatizer = WordNetLemmatizer()
nlp = spacy.load('en_core_web_sm')

vocabulary = [
    'FOI','vapes','vape','riley'
]
custom_stopwords = {
    'freedom','information','act','following','report',
    'please','dear','yes','request','provide',
    'hammersmith','fulham','like','email','requesting',
    'see','attach','foi','attached','service','would','many',
    'faithfully','number','council','authority','local','year',
    'borough','name','could','data','within','question','including',
    'copy','also','made','regarding','since','relating','requested',
    'thank','confirm','sent','make','date','much','kind','regard',
    'good','afternoon','evening','sir','madam','hello','per','correspondence',
    'address','total','contact','look','forward','hearing','way','day','apologise',
    'responding','grateful','necessary','period','list','time','related','detail',
    'use','end','last'
}
month_stopwords = {
    'january','february','march','april',
    'may','june','july','august','september',
    'october','november','december','jan','feb',
    'mar','apr','may','jun','jul','aug','sep',
    'oct','nov','dec','sept'
}

spell.word_frequency.load_words(vocabulary)
for word in vocabulary:
    custom_segmenter.unigrams[word] = 1e9
stop_words = set(stopwords.words('english'))
stop_words.update(custom_stopwords)
stop_words.update(month_stopwords)

import gensim
from gensim.models import CoherenceModel#, LdaModel
from gensim.corpora import Dictionary

import pyLDAvis.gensim_models as gensimvis
import pickle
import pyLDAvis

# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.decomposition import LatentDirichletAllocation

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\jf79\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jf79\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jf79\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Standard Functions

In [4]:
# Standard Functions #

# Function to clean labels in any plot functions
def clean_label(label):
    return label.replace('_', ' ').title()
 
# Function for getting the name of a Dataframe
def get_var_name(var):
    for name, value in globals().items():
        if value is var:
            return name
 
# Function to validate the data in a Dataframe
def validate_data(df, show_counts=True):
    df_name = get_var_name(df)
    print(f'#########################################################################################################################################################################################\nDataFrame: {df_name}')
    #snapshot the dataset
    display(df)
    #check for unique values
    unique_counts = pd.DataFrame(df.nunique())
    unique_counts = unique_counts.reset_index().rename(columns={0:'No. of Unique Values', 'index':'Field Name'})
    print("Unique values per field:")
    pd.set_option('display.max_rows', None)
    display(unique_counts)
    pd.reset_option('display.max_rows')
    #checking for duplicates
    duplicate_count = df.duplicated().sum()
    print("\nNumber of duplicate rows:")
    print(duplicate_count,'\n')
    info = df.info(show_counts=show_counts)
    display(info)
    #summary stats
    print("\nSummary statistics:")
    display(df.describe())
    print('End of data validation\n#########################################################################################################################################################################################\n')
 
# Function to provide list for data sources as a dataframe when conducting analysis
def header_list(df):
    df_list_ = df.copy()
    df_list = df_list_.columns.tolist()
    df_list = pd.DataFrame(df_list)
    new_header = df_list.iloc[0]  # Get the first row for the header
    df_list = df_list[1:]  # Take the data less the header row
    df_list.columns = new_header  # Set the header row as the df header
    df_list.reset_index(drop=True, inplace=True)  # Reset index
   
    return df_list
 
def query_data(schema, data):
    # Define the SQL query
    query = f'SELECT * FROM [{schema}].[{data}]'
 
    # Load data into DataFrame
    df = pd.read_sql(query, engine)
 
    print(f'Successfully imported {data}')
    # Display the DataFrame
    return df

def display(df):
    # Attempt to get the name of the DataFrame from the caller's local variables
    frame = inspect.currentframe().f_back
    # Attempt to find the variable name corresponding to the DataFrame
    name = "Unnamed DataFrame"
    for var_name, var_value in frame.f_locals.items():
        if var_value is df:
            name = var_name
            break
 
    # If the name is not in the list to be excluded, print it
    if name not in {'df', 'Unnamed DataFrame', 'unique_counts'}:
        print(f"DataFrame: {name}")
    # Always display the DataFrame regardless of the name
    original_display(df)

def unique_values(df, display_df=True):
    # Extract unique values for each field and store them in a dictionary
    unique_values = {col: df[col].unique() for col in df.columns}
    # Find the maximum number of unique values
    max_length = max(len(values) for values in unique_values.values())
    # Create a dictionary for the new DataFrame with padded None values
    unique_df_data = {}
    for col, values in unique_values.items():
        unique_df_data[col] = list(values) + [None] * (max_length - len(values))
    # Create the new DataFrame
    unique_df = pd.DataFrame(unique_df_data)
    if display_df == True:
        # Set display options to show all rows and display the DataFrame
        pd.set_option('display.max_rows', None)
        display(unique_df.head(100))
        # Reset display options back to default
        pd.reset_option('display.max_rows')

def read_directory():
    directory = os.getcwd()
    files = os.listdir(os.getcwd())
    print(f"Your Current Directory is: {directory}")
    print("Files in: %s" % (files))

def export_to_csv(df):
    df_name = get_var_name(df)
    # Specify the directory and filename
    directory = r"C:\Users\jf79\OneDrive - Office Shared Service\Documents\H&F Analysis\Python CSV Repositry"
    file_path = f'{directory}\\{df_name}.csv'
    # Export the DataFrame to the specified directory
    df.to_csv(file_path, index=False)
    print(f'Successfully exported {df_name} to CSV')

Script Specific Functions

In [5]:
# Script Specific Functions #
def get_pos_tags(text):
    doc = nlp(text)
    return {token.text: token.tag_ for token in doc}

def is_valid_correction(original, pos_tags):
    # Skip correction if the original word matches these patterns
    if re.match(r"\d+(st|nd|rd|th)", original):  # Ordinal numbers
        return False
    if pos_tags and pos_tags.get(original) == 'NNP' or pos_tags.get(original) == 'PROPN':
        return False
    return True

def is_valid_token(word):
    if len(word) > 2:
        return True
    if any(char.isdigit() for char in word):
        return False
    return False

# Tokenization function
def safe_tokenize(text):
    if isinstance(text, str):
        return word_tokenize(text)
    return []

def segementing_and_spelling(tokens):
    # Step 3: Spelling correction and word segmentation
    corrected_tokens = []
    for word in tokens:
        split_words = custom_segmenter.segment(word)  # Split merged words
        for subword in split_words:
            corrected_word = spell.correction(subword) or word # Correct spelling
            pos_tags = get_pos_tags(subword)
            if not is_valid_correction(subword, pos_tags):  # Skip invalid corrections
                corrected_word = subword
            corrected_tokens.append(corrected_word)
        
    filtered_tokens = [word for word in corrected_tokens if is_valid_token(word)]

    
    return filtered_tokens

def process_text(row,details):
    text = row[f'{details}']
    if not isinstance(text, str):  # Convert non-string types to string
        text = str(text)
    if not text.strip():  # Skip empty or whitespace strings
        return ""
 
    try:
        # Step 1: Text cleaning
        text = text.lower()  # Lowercase
        text = re.sub(r"https?://\S+|www\.\S+", "", text)  # Remove URLs
        text = re.sub(r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\b", "", text)  # Remove email addresses
        text = re.sub(r"\d+", "", text)  # Remove numbers
        text = re.sub(r"[^\w\s]", "", text)  # Remove punctuation
        text = re.sub(r"\s+", " ", text).strip()  # Normalize spaces
 
        # Step 2: Tokenization
        tokens = text.split()
        
        # Step 3 & 4: Word segmenting and spelling
        corrected_tokens = segementing_and_spelling(tokens=tokens)
 
        # Step 5: Lemmatization and stop word removal
        lemmatized_tokens = [lemmatizer.lemmatize(word) for word in corrected_tokens if word not in stop_words]

        lemmatized_tokens = [word for word in lemmatized_tokens if word not in stop_words]

        # Step 6: Word segmenting and spelling (again)
        final_tokens = segementing_and_spelling(tokens=lemmatized_tokens)

        final_tokens = [word for word in final_tokens if word not in stop_words]

        # Step 7: Join processed tokens into a string
        return " ".join(final_tokens)
 
    except Exception as e:
        print(f"Error processing text: {repr(text)}")
        raise e
 
 
def process_text_with_progress(df,details):
    # Initialize tqdm progress bar
    pbar = tqdm.tqdm(total=len(df), desc="Processing rows", unit="row", dynamic_ncols=True)
    def process_row(row):
        try:
            return process_text(row,details=details)
        finally:
            # Update the progress bar for every row processed
            pbar.update(1)
    
    df['processed_details'] = df.apply(process_row, axis=1)
    df['processed_tokens'] = df['processed_details'].apply(safe_tokenize)

    pbar.close()
    
    return df

def prepare_corpus(tokens, threshold=100, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    # Generate bigrams and trigrams
    bigram = gensim.models.Phrases(tokens, min_count=5, threshold=threshold)
    trigram = gensim.models.Phrases(bigram[tokens], threshold=threshold)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)

    bigram_texts = (bigram_mod[doc] for doc in tokens)
    trigram_texts = (trigram_mod[bigram_mod[doc]] for doc in bigram_texts)

    # Verify POS of each word
    texts_out = []
    for text in trigram_texts:
        # Use SpaCy to process the text
        doc = nlp(" ".join(text))  # Join tokens into a string and process with SpaCy
        filtered_tokens = [token.text for token in doc if token.pos_ in allowed_postags]
        texts_out.append(filtered_tokens)
    return texts_out

def count_words(tokens):
    # Flatten the tokenized details
    all_tokens = [token for sublist in tokens for token in sublist]
    # Count word frequencies
    word_counts = Counter(all_tokens)
    # Print the top 10 most common words
    word_counts = pd.DataFrame([word_counts])
    word_counts.reset_index(inplace=True)
    word_counts = word_counts.melt(id_vars='index',var_name='Word',value_name='Count')
    word_counts.sort_values(by='Count',ascending=False,inplace=True)
    word_counts.reset_index(inplace=True,drop=['index'])
    word_counts.drop(columns='index',inplace=True)
    display(word_counts.head(25))
    display(word_counts)

# Model Evaluation Function
def compute_coherence_values(corpus, dictionary, k, a, b):
    lda_model = gensim.models.LdaMulticore(
        corpus=corpus,
        id2word=dictionary,
        num_topics=k,
        random_state=42,
        chunksize=100,
        passes=10,
        alpha=a,
        eta=b,
        workers=4
    )
    coherence_model_lda = CoherenceModel(model=lda_model, texts=FOI_corpus, dictionary=id2words, coherence='c_v')

    return coherence_model_lda.get_coherence()

# Define the function for computing coherence for parameter sets
def compute_for_params(corpus_set, k, a, b, corpus_title):
    try:
        # Thread-local coherence value computation
        cv = compute_coherence_values(corpus=corpus_sets[corpus_set], dictionary=id2words, k=k, a=a, b=b)
        # The result is created with thread-local variables
        result = {
            'Validation_Set': corpus_title,
            'Topics': k,
            'Alpha': a,
            'Beta': b,
            'Coherence': cv
        }
        results_queue.put(result)
    except Exception as e:
        print(f"Error processing (k={k}, a={a}, b={b}, corpus_title={corpus_title}): {str(e)}")

def run_io_task_in_parallel(tasks):
    with ThreadPoolExecutor(max_workers=4) as executor:
        futures = [executor.submit(task) for task in tasks]
        for future in futures:
            future.result()

def find_element(corpus_title):
    if corpus_title == '75% Corpus':
        return 0
    return 1

def sankey_plot(df, source, target, value, name='Value'):
    # # Count the occurrences of each unique pair
    # counts = df.groupby([f'{source}',f'{target}']).size().reset_index(name='Value')
    
    # # Create a mapping of unique labels to indices
    # unique_labels = list(set(counts[f'{source}'].tolist() + counts[f'{target}'].tolist()))
    # label_to_index = {label: i for i, label in enumerate(unique_labels)}
    
    # # Map source and target to their indices
    # counts['sourceIndex'] = counts[f'{source}'].map(label_to_index)
    # counts['targetIndex'] = counts[f'{target}'].map(label_to_index)
    
    # # Prepare data for Sankey diagram
    # source = counts['sourceIndex'].tolist()
    # target = counts['targetIndex'].tolist()
    # value = counts['Value'].tolist()
    
    # Step 1: Aggregate the data by summing the 'value' for each (source, target) pair
    agg_df = df.groupby([f'{source}', f'{target}'])[f'{value}'].sum().reset_index()
    display(agg_df)
    # Step 2: Create a list of unique sources and targets
    all_nodes = list(set(agg_df[f'{source}']).union(set(agg_df[f'{target}'])))
    
    # Step 3: Create dictionaries to map nodes to indices
    source_indices = {node: i for i, node in enumerate(all_nodes)}
    target_indices = {node: i for i, node in enumerate(all_nodes)}

    # Step 4: Prepare data for the Sankey diagram
    # We need to convert the sources and targets to their corresponding indices
    sankey_data = {
        'source': agg_df[f'{source}'].map(source_indices),
        'target': agg_df[f'{target}'].map(target_indices),
        'value': agg_df[f'{value}']
    }

    # Create Sankey diagram
    fig = go.Figure(data=[go.Sankey(
        node=dict(
            pad=7,
            thickness=20,
            line=dict(color="black", width=0.01),
            label=all_nodes
        ),
        link=dict(
            source=sankey_data['source'],
            target=sankey_data['target'],
            value=sankey_data['value']
        )
    )])
    
    fig.update_layout(
        title_text="Sankey Diagram", font_size=10,
        width=1000,
        height=800    
    )
    fig.show()

    fig.write_html(f'sankey_diagram {name}.html')

def sankey3_diagram(df, source, intermediate, target, value, name='Value'):
    # Step 1: Group the data to aggregate the values for the flows
    df_grouped = df.groupby([source, intermediate, target], as_index=False)[value].sum()
 
    # Step 2: Create a list of unique nodes for all three stages (source, intermediate, target)
    all_nodes = list(set(df_grouped[source]).union(set(df_grouped[intermediate])).union(set(df_grouped[target])))
    # Step 3: Create dictionaries to map nodes to indices
    node_indices = {node: i for i, node in enumerate(all_nodes)}
    # Step 4: Prepare data for the Sankey diagram
    # Map the 'source', 'intermediate', and 'target' fields to their corresponding indices
    df_grouped['source_idx'] = df_grouped[source].map(node_indices)
    df_grouped['intermediate_idx'] = df_grouped[intermediate].map(node_indices)
    df_grouped['target_idx'] = df_grouped[target].map(node_indices)
    # Step 5: Prepare the flow data for the Sankey diagram
    # First, create the flows from source to intermediate
    links_source_intermediate = pd.DataFrame({
        'source': df_grouped['source_idx'],
        'target': df_grouped['intermediate_idx'],
        'value': df_grouped[value]
    })
    # Then, create the flows from intermediate to target
    links_intermediate_target = pd.DataFrame({
        'source': df_grouped['intermediate_idx'],
        'target': df_grouped['target_idx'],
        'value': df_grouped[value]
    })
    # Combine the two sets of links
    all_links = pd.concat([links_source_intermediate, links_intermediate_target], ignore_index=True)
    # Step 6: Create the Sankey diagram using plotly
    fig = go.Figure(go.Sankey(
        node=dict(
            pad=15,
            thickness=20,
            line=dict(color="black", width=0.01),
            label=all_nodes
        ),
        link=dict(
            source=all_links['source'],
            target=all_links['target'],
            value=all_links['value']
        )
    ))
    fig.update_layout(
        title_text="Sankey Diagram", font_size=10,
        width=1200,
        height=1000    
    )
    # Show the plot
    fig.show()

    fig.write_html(f'sankey_diagram {name}.html')

In [ ]:
cd

Database and CWD setup and connection

In [6]:
# Database credentials
db_host = get_env('DB_HOST')
db_port = '1433'
db_name = 'IA_ODS'
 
# Create the connection string for SQL Server using pyodbc with Windows Authentication
connection_string = f'mssql+pyodbc://@{db_host}:{db_port}/{db_name}?driver=ODBC+Driver+17+for+SQL+Server&Trusted_Connection=yes'

# Create the database engine
engine = create_engine(connection_string)

# Define the current working directory
cwd = r'C:\Users\jf79\OneDrive - Office Shared Service\Documents\H&F Analysis\Fuel Poverty Analysis\Fuel Poverty General\Fuel Bible'
os.chdir(cwd)
files = os.listdir(os.getcwd())
print("Files in %r: %s" % (cwd, files))

Files in 'C:\\Users\\jf79\\OneDrive - Office Shared Service\\Documents\\H&F Analysis\\Fuel Poverty Analysis\\Fuel Poverty General\\Fuel Bible': ['certificates.csv', 'columns.csv', 'LICENCE.txt', 'recommendations.csv', 'schema.json', 'UPRN_to_LSOA_HF.csv']


In [7]:
# Read the CSV file into a DataFrame
cwd = r'C:\Users\jf79\OneDrive - Office Shared Service\Documents\H&F Analysis\FOI Sentiment Analysis\Data'
os.chdir(cwd)
read_directory()

FOI_export = pd.read_csv('FOI_export.csv')
FOI_details_data = pd.read_excel('FOI Details Data (2024).xlsx')
FOI_email_data = pd.read_excel('FOI Email Data.xlsx')

Your Current Directory is: C:\Users\jf79\OneDrive - Office Shared Service\Documents\H&F Analysis\FOI Sentiment Analysis\Data
Files in: ['Coherence Values.csv', 'FOI Details Data (2024).csv', 'FOI Details Data (2024).xlsx', 'FOI Details Data.csv', 'FOI Email Data.xlsx', 'FOI_export.csv', 'lda_tuning_results.csv', 'sankey_diagram DIRECTORATE.html', 'sankey_diagram Value.html', 'sankey_diagram.html']


In [8]:
FOI_final = FOI_export.copy()
FOI_details = FOI_details_data.copy()
FOI_email = FOI_email_data.copy()
FOI_final.columns = FOI_final.columns.str.lower().str.strip().str.replace(' ','_')
FOI_details.columns = FOI_details.columns.str.lower().str.strip().str.replace(' ','_')
FOI_details.rename(columns={'caseid':'caseid_final'},inplace=True)
# FOI_final = FOI_final[FOI_final['processed_details'] != '']
FOI_final['processed_tokens'] = FOI_final['processed_details'].apply(safe_tokenize)
FOI_final['new_tokens'] = FOI_final['processed_tokens'].apply(lambda x: [word for word in x if word not in stop_words])
FOI_final = pd.merge(
    FOI_final, FOI_details,
    how='left', left_on='caseid', right_on='caseid_final',
    suffixes=('', '_drop')
)
FOI_final.drop([col for col in FOI_final.columns if '_drop' in col], axis=1, inplace=True)
# count_words(FOI_final['new_tokens'])
FOI_final = FOI_final[FOI_final['caseid_final'].isna() == False]
FOI_final['email'] = FOI_final['email'].fillna('')
# Function to extract the domain part
def extract_domain(email):
    try:
        email = email.lower()
        match = re.search(r'@([\w.-]+)', email)
        if match:
            return match.group(1)
        return None
    except Exception as e:
        print(f"Error extracting domain from email: {email}")
        raise e
# Apply the function to the email column
FOI_final['domain'] = FOI_final['email'].apply(extract_domain)
FOI_final[['group','domain']] = FOI_final['domain'].str.split('.',n=1, expand=True)
columns_to_drop = ['processed_tokens','new_tokens']
columns = [x for x in FOI_final.columns.to_list() if x not in columns_to_drop]
FOI_final = pd.merge(
    FOI_final, FOI_email,
    how='left', left_on='group', right_on='domain',
    suffixes=('', '_drop')
)
FOI_final['category'] = FOI_final['category'].fillna('Unknown') 
# export_to_csv(FOI_final)

In [9]:
import plotly.graph_objects as go
 
# Count the occurrences of each unique pair
counts = FOI_final.groupby(['category', 'directorate']).size().reset_index(name='Value')
 
# Create a mapping of unique labels to indices
unique_labels = list(set(counts['category'].tolist() + counts['directorate'].tolist()))
label_to_index = {label: i for i, label in enumerate(unique_labels)}
 
# Map category and directorate to their indices
counts['categoryIndex'] = counts['category'].map(label_to_index)
counts['directorateIndex'] = counts['directorate'].map(label_to_index)
 
# Prepare data for Sankey diagram
category = counts['categoryIndex'].tolist()
directorate = counts['directorateIndex'].tolist()
value = counts['Value'].tolist()
 
# Create Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=7,
        thickness=20,
        line=dict(color="black", width=0.01),
        label=unique_labels
    ),
    link=dict(
        source=category,
        target=directorate,
        value=value
    )
)])
 
fig.update_layout(
    title_text="Sankey Diagram", font_size=10,
    width=1000,
    height=800    
)
fig.show()

fig.write_html('sankey_diagram.html')

In [10]:
FOI_corpus = prepare_corpus(FOI_final['new_tokens'],threshold=50)

# Create Dictionary
id2words = Dictionary(FOI_corpus)

# Term Document Frequency
corpus = [id2words.doc2bow(text) for text in FOI_corpus]

In [11]:
# # Initialize variables
# results = pd.DataFrame(columns=['Validation_Set', 'Topics', 'Alpha', 'Beta', 'Coherence'])
# lock = threading.Lock()  # Thread-safe lock

# # Number of documents in the corpus
# num_of_docs = len(corpus)

# # Prepare corpus sets (75% and 100% corpus)
# corpus_sets = [
#     gensim.utils.ClippedCorpus(corpus, int(num_of_docs * 0.75)),
#     corpus
# ]

# # Hyper-parameters
# min_topics = 2
# max_topics = 15
# step_size = 1
# topics_range = list(range(min_topics,max_topics+1,step_size))
# alphas = np.round(np.linspace(0.01, 0.91, 5), 2).tolist() + ['symmetric', 'asymmetric']
# betas = np.round(np.linspace(0.01, 0.91, 5), 2).tolist() + ['symmetric']
# corpus_titles = ['75% Corpus', '100% Corpus']

# # Define test combinations to run in parallel
# test_combinations = [
#     (find_element(corpus_title),topic,alpha,beta,corpus_title)
#     for topic,alpha,beta,corpus_title, in itertools.product(topics_range,alphas,betas,corpus_titles)
# ]

# tasks = [lambda combination=combination: compute_for_params(*combination) for combination in test_combinations]
# # Process tasks in parallel with a progress bar
# with tqdm(total=len(test_combinations), desc='Processing combinations') as pbar:
#     def task_with_progress(task):
#         task()
#         pbar.update(1)
    
#     run_io_task_in_parallel([lambda task=task: task_with_progress(task) for task in tasks])

# while not results_queue.empty():
#     result = results_queue.get()
#     results = pd.concat([results, pd.DataFrame([result])], ignore_index=True)

# display(results)
# export_to_csv(results)

In [12]:
num_topics = 6
# Build LDA model
lda_model = gensim.models.LdaMulticore(
    corpus=corpus,
    id2word=id2words,
    num_topics=num_topics,
    random_state=42,
    chunksize=100,
    passes=10,
    alpha=0.24,
    eta=0.91
)

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=FOI_corpus, dictionary=id2words, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)
# Assuming lda_model is your trained LdaModel
lda_model.save(f"C:/Users/jf79/OneDrive - Office Shared Service/Documents/H&F Analysis/FOI Sentiment Analysis/Models/lda_model ({num_topics})")

Coherence Score:  0.32146038579067077


In [14]:
# Visualize the topics
pyLDAvis.enable_notebook()
path = 'C:/Users/jf79/OneDrive - Office Shared Service/Documents/H&F Analysis/FOI Sentiment Analysis/Data'
LDAvis_data_filepath = os.path.join(path+str(num_topics))

# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2words)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

pyLDAvis.save_html(LDAvis_prepared,path+str(num_topics)+'.html')

LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.063599 -0.010010       1        1  36.418688
5     -0.001962  0.067715       2        1  17.462923
0      0.029713 -0.015203       3        1  12.659116
2     -0.031875  0.003332       4        1  12.532629
4     -0.014223 -0.024064       5        1  11.981696
1     -0.045252 -0.021770       6        1   8.944947, topic_info=          Term        Freq       Total Category  logprob  loglift
409   contract  234.000000  234.000000  Default  30.0000  30.0000
502   property  203.000000  203.000000  Default  29.0000  29.0000
309      child  330.000000  330.000000  Default  28.0000  28.0000
599  placement   84.000000   84.000000  Default  27.0000  27.0000
301     school  198.000000  198.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
24     include    8.353642  102.372737   Topic6  -6.2612  -0.0918
248  available    8.068600  119.808704   Topic6  -6.2959  -0.2838
395      month    7.179096  130.019942   Topic6  -6.4127  -0.4824
63         beg    6.832869   65.165397   Topic6  -6.4621   0.1589
115       note    6.727756   52.082444   Topic6  -6.4776   0.3675

[413 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
917       1  0.053160  academic
917       2  0.053160  academic
917       3  0.744243  academic
917       4  0.053160  academic
917       5  0.053160  academic
...     ...       ...       ...
2138      2  0.057210     youth
2138      3  0.114420     youth
2138      4  0.629312     youth
2138      5  0.057210     youth
2138      6  0.057210     youth

[1682 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 6, 1, 3, 5, 2])

In [16]:
lambd = 0.2  # A specific relevance metric value
 
all_topics = {}
all_topics_with_docs = {}
num_topics = lda_model.num_topics
num_terms = 30
num_docs = 5  # Number of most related documents to retrieve for each topic
 
topic_doc_matrix = lda_model.get_document_topics(corpus, minimum_probability=0)  # Get the topic-document distribution
print
for i in range(1, num_topics + 1):  # Indices are 1-based, not 0-based
    # Extract the term information
    topic = LDAvis_prepared.topic_info[LDAvis_prepared.topic_info.Category == 'Topic' + str(i)].copy()
    topic['relevance'] = topic['loglift'] * (1 - lambd) + topic['logprob'] * lambd
    all_topics['Topic ' + str(i)] = topic.sort_values(by='relevance', ascending=False).Term[:num_terms].values
 
    # Find the most related documents for the topic
    topic_index = i - 1  # Convert to 0-based index
    doc_relevance = [doc[topic_index][1] for doc in topic_doc_matrix]  # Extract the relevance of documents for this topic
    top_doc_indices = sorted(range(len(doc_relevance)), key=lambda x: doc_relevance[x], reverse=True)
    sorted_doc_relevance = [doc_relevance[i] for i in top_doc_indices]
    
    # Retrieve the actual document texts
    top_docs_content = [FOI_final['details'][idx] for idx in top_doc_indices]  # Assuming `documents` contains the original texts
    all_topics_with_docs['Topic ' + str(i)] = {
        'terms': all_topics['Topic ' + str(i)],
        'top_docs_indices': top_doc_indices,  # Indice of most relevant doc
        'top_doc_relevance': sorted_doc_relevance,  # relevance of doc
        # 'top_docs': top_docs_content  # Contents of the most relevant documents
    }
 
# Create a DataFrame to display terms and documents for each topic
output_data = {
    'Topic': [],
    'Top Terms': [],
    'Top Doc Indices': [],
    'Top Doc Relevance': [],
    # 'Top Documents': []
}

for topic, data in all_topics_with_docs.items():
    output_data['Topic'].append(topic)
    output_data['Top Terms'].append(", ".join(data['terms']))
    output_data['Top Doc Indices'].append(", ".join(map(str,data['top_docs_indices'])))
    output_data['Top Doc Relevance'].append(", ".join(map(str,data['top_doc_relevance'])))
    # output_data['Top Documents'].append(" || ".join(data['top_docs']))  # Separate documents with '||'

output_data = pd.DataFrame(output_data)
export_to_csv(output_data)

Successfully exported output_data to CSV


In [17]:
output = output_data.copy()
FOI_topics = FOI_final.copy()
# Step 1: Convert 'col1' and 'col2' into lists
output['Top Doc Indices List'] = output['Top Doc Indices'].str.split(',')
output['Top Doc Relevance List'] = output['Top Doc Relevance'].str.split(',')

# Step 2: Create a new column that combines both lists into a dictionary
output['combined_dict'] = output.apply(lambda row: dict(zip(row['Top Doc Indices List'], row['Top Doc Relevance List'])), axis=1)
output.drop(['Top Doc Indices List','Top Doc Relevance List','Top Doc Indices','Top Doc Relevance', 'Top Terms'],axis=1,inplace=True)
output.set_index('Topic',inplace=True)

expanded_rows = output['combined_dict'].apply(pd.Series).stack().reset_index()
expanded_rows['index'] = expanded_rows['level_1'].astype('Int64')

columns = [
    'caseid',
    'directorate',
    'category'
]
FOI_topics = FOI_topics[columns]
FOI_topics.reset_index(inplace=True)
FOI_merged = pd.merge(
    expanded_rows, FOI_topics,
    how='left', on='index',
    suffixes=('', '_drop')
)
FOI_merged.drop(columns=['level_1','index'],inplace=True)
FOI_merged.rename(columns={0:'Relevance'},inplace=True)
FOI_merged['Relevance'] = FOI_merged['Relevance'].astype(float)

topic_names = {
    'Topic 1' : 'Housing and Support',
    'Topic 2' : 'Contract Management',
    'Topic 3' : 'Education and Schools',
    'Topic 4' : 'Legal and Regulatory',
    'Topic 5' : 'Retail and Consumer Issues',
    'Topic 6' : 'Property and Business'
}
FOI_merged['Topic'] = FOI_merged['Topic'].map(topic_names)
FOI_merged
sankey3_diagram(FOI_merged,source='category',intermediate='Topic',target='directorate', value='Relevance', name='DIRECTORATE')

In [ ]:
topic_doc_matrix = lda_model.get_document_topics(corpus, minimum_probability=0)  # Get the topic-document distribution
for i in range(1, num_topics + 1):  # Indices are 1-based, not 0-based
    # Find the most related documents for the topic
    topic_index = i - 1  # Convert to 0-based index
    doc_relevance = [doc[topic_index][1] for doc in topic_doc_matrix]  # Extract the relevance of documents for this topic
    top_doc_indices = sorted(range(len(doc_relevance)), key=lambda x: doc_relevance[x], reverse=True)[:num_docs]
    print(doc_relevance[0])
    print(f'{topic_index}: {topic_doc_matrix[0][topic_index][1]}')

In [ ]:
doc_relevance = [doc[topic_index][1] for doc in topic_doc_matrix]
topic_doc_matrix[0][0][1]